# SLSTR Composite Plotter

    Version: 1.0
    Date:    27/09/2019
    Author:  Ben Loveday (Plymouth Marine Laboratory) and Hayley Evers-King (EUMETSAT)
    Credit:  This code was developed for EUMETSAT under contracts for the Copernicus 
             programme.
    License: This code is offered as open source and free-to-use in the public domain, 
             with no warranty.

**What is this notebook for?**

This notebook will plot a Sentinel-3 SLSTR SST image composite by overlayinf three scenes.

**What specific tools does this notebook use?**

Image processing toolkit

***

In [ ]:
import os
import shutil
import sys
import warnings
import datetime
import numpy as np
import fnmatch
import logging
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import gridspec
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.patheffects as path_effects
import xmltodict
import glob
import warnings
warnings.filterwarnings("ignore")

# specific tools (which can be found here ../../Hub_tools/)
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())) + '/Hub_Tools/')
import image_tools as img

***

### Plotting function

In this case, we plot our SLSTR scenes using a function that manages data ingestion, flagging, bias correction and makes some map embelishements (e.g. adds dotted lines to the scene edges, so we can tell where the boundaries are). We call this function for each image in the boxes further down.

In [ ]:
def make_SLSTR_composite_plot(nc_files, plot_extents=None, fsz=20,\
                              land_resolution='50m', vmin=10, vmax=28,\
                              xsize=20, ysize=16, dpi=150, QMASK=4,\
                              cmap=plt.cm.RdYlBu_r):
    '''
     Plots SLSTR images from n input files, will overay first to last.
    '''
    # get cartopy land layers
    land_poly = cfeature.NaturalEarthFeature('physical', 'land', land_resolution,
                                             edgecolor='k',
                                             facecolor=cfeature.COLORS['land'])
    
    # setup figure
    fig = plt.figure(figsize=(xsize, ysize), dpi=dpi)
    plt.rc('font', size=fsz)
    
    # setup axes
    gs = gridspec.GridSpec(3, 1, height_ratios=[20,0.5,1])
    gs.update(wspace=0.01, hspace=0.01)

    # setup plot 1
    axes_m = plt.subplot(gs[0,0], projection=ccrs.PlateCarree())
    axes_m.background_patch.set_facecolor('0.5')


    if plot_extents:
        axes_m.set_extent(plot_extents, ccrs.PlateCarree())

    # read data and plot background
    for nc_file in nc_files:
        nc_fid = xr.open_dataset(nc_file)
        LON = nc_fid.lon.data
        LAT = nc_fid.lat.data
        
        x1 = 0 ; x1 = -1; y0 = 0 ; y1 = -1
        # get subset based on plot extents
        if plot_extents:
            x1, x2, y1, y2 = img.subset_image(LAT, LON, plot_extents)

        # read in and subset if required
        LON = LON[x1:x2,y1:y2]
        LAT = LAT[x1:x2,y1:y2]
        SST_raw = np.squeeze(nc_fid.sea_surface_temperature.data[0,x1:x2,y1:y2])
        SST_BIAS = np.squeeze(nc_fid.sses_bias.data[0,x1:x2,y1:y2])
        QUALITY_LEVEL = np.squeeze(nc_fid.quality_level.data[0,x1:x2,y1:y2])
        nc_fid.close()
        
        # correct SST
        SST_C = SST_raw + SST_BIAS - 273.15

        # flag data
        SST_C[QUALITY_LEVEL<=QMASK] = np.nan

        # plot the SST field
        p1 = axes_m.pcolormesh(LON, LAT, SST_C, cmap=cmap,\
                          vmin=vmin, vmax=vmax, zorder=-1)

        # add the plot edges
        xml_file = os.path.dirname(nc_file)+'/xfdumanifest.xml'
        with open(xml_file) as fd:
            doc = xmltodict.parse(fd.read())
            coords = doc['xfdu:XFDU']['metadataSection']['metadataObject'][2]\
                        ['metadataWrap']['xmlData']['sentinel-safe:frameSet']\
                        ['sentinel-safe:footPrint']['gml:posList']

        # split the coords
        lats_lons = coords.split(' ')
        lats = np.asarray(lats_lons[::2]).astype(float)
        lons = np.asarray(lats_lons[1::2]).astype(float)

        if 'S3B' in nc_file:
            plot_col = 'b'
        else:
            plot_col = 'k'

        p2 = axes_m.plot(lons, lats, c=plot_col, linewidth=1,\
                    linestyle='--', zorder=5, alpha=0.5,\
                    transform=ccrs.PlateCarree())

    # add some map embelishments
    axes_m.coastlines(resolution=land_resolution, color='black', linewidth=1)
    axes_m.add_feature(land_poly)
    g1 = axes_m.gridlines(draw_labels = True)
    g1.xlabels_top = False
    g1.ylabels_right = False

    g1.xlabel_style = {'size': fsz, 'color': 'gray'}
    g1.ylabel_style = {'size': fsz, 'color': 'gray'}

    # setup plot2: colorbar
    axes_c = plt.subplot(gs[2,0])
    cbar = plt.colorbar(p1, cax=axes_c, orientation='horizontal')
    cbar.ax.tick_params(labelsize=fsz) 
    cbar.set_label('SLSTR SST composite [$^{o}$C]',fontsize=fsz)
    
    return gs, axes_m, axes_c

### This is where our main code starts...

We start by defining our data paths and finding all the necessary files (which glob.glob takes care of). Ths files are added to a list which is then sent to our large function above.

In [ ]:
# where the data should be downloaded to:
download_dir_path = "/home/jovyan/work/products"

# verbosity
verbose = False

# figure options
figure_font_size = 20
plot_extents = [-160, -116, 10, 45]
vmin = 10
vmax = 28

# get the files
SLSTR_files = glob.glob(os.path.join(download_dir_path,'S3*WST*201909*','*.nc'))

If we set verbose=True in the box above box below will print out the files in has found.

In [ ]:
if verbose:
    for SLSTR_file in SLSTR_files:
        print(SLSTR_file)

And now we pass our list of files to the plotting routine. The plotting routine returns the handles of our axes, so that we can still make some changes once the main plot is done (e.g. add the annotations). Finally, it will save fie figure.

In [ ]:
# make the plot: we will call this as a function as in contains a 'for' loop to make the plot
fig, axis, colbar = make_SLSTR_composite_plot(SLSTR_files, plot_extents=plot_extents,\
                                              fsz=figure_font_size, vmin=vmin, vmax=vmax)

# add some embellishments
plt.sca(axis)

label='Sentinel-3A SLSTR SST\n07/09/2019 (22:27 local time)'    
txt = plt.annotate(label, xy=(0.66, 1.01), xycoords='axes fraction',\
               size=figure_font_size/1.25,
               color='k', zorder=100, annotation_clip=False)

label='Sentinel-3A SLSTR SST\n08/09/2019 (00:08 local time)'    
txt = plt.annotate(label, xy=(0.005, 1.01), xycoords='axes fraction',\
               size=figure_font_size/1.25,
               color='k', zorder=100, annotation_clip=False)

label='Sentinel-3B SLSTR SST\n08/09/2019 (23:28 local time)'    
txt = plt.annotate(label, xy=(0.33, 1.01), xycoords='axes fraction',\
               size=figure_font_size/1.25,
               color='b', zorder=100, annotation_clip=False)

plt.savefig('SLSTR_All_SST_California_20190909.png',bbox_inches='tight')